# I2k Workshop 2024 - Exploiting NanoPyx’s Liquid Engine to accelerate image analysis pipelines 

## NanoPyx

<img src="https://github.com/HenriquesLab/NanoPyx/raw/main/.github/logo.png" align="right" width="300"/>

### What is the NanoPyx 🔬 Library?

NanoPyx is a library specialized in the analysis of light microscopy and super-resolution data.
It is a successor to [NanoJ](https://github.com/HenriquesLab/NanoJ-Core), which is a Java library for the analysis of super-resolution microscopy data.

NanoPyx focuses on performance, by using the [Liquid Engine](https://github.com/HenriquesLab/LiquidEngine) at its core.

The source code documentation for nanopyx can be found [here](https://henriqueslab.github.io/NanoPyx/nanopyx.html).

## Liquid Engine

The Liquid Engine is a high-performance, adaptive framework designed to optimize computational workflows for bioimage analysis. It dynamically generates optimized CPU and GPU-based code variations and selects the fastest combination based on input parameters and device performance, significantly enhancing computational speed. The Liquid Engine employs a machine learning-based Agent to predict the optimal combination of implementations, adaptively responding to delays and performance variations.

## In this tutorial:

1. We will showcase basic examples of NanoPyx methods
2. We will use the Liquid Engine in the NanoPyx library
3. We will implement our own custom method in the Liquid Engine

# 1. Using NanoPyx methods

## 1.1 Install NanoPyx and it's dependencies

NanoPyx is available through [PyPi](https://pypi.org/) so it can simply be pip installed:
```shell
!pip install nanopyx
```

To be used in a local jupyter notebook it is recommended that the optional dependencies for jupyter are installed:
```shell
!pip install nanopyx[jupyter]
```

If using Google Colab install the colab dependencies:
```shell
!pip install nanopyx[colab]
```

In [1]:
# your code here
!pip install nanopyx[colab] -q

## 1.2 Let's use NanoPyx to load an example image

NanoPyx contains several example image stacks that can be used to try it's different methods.
You can access them through the following class:
```python
from nanopyx.data.download import ExampleDataManager
```

a) Let's import the class and list all available datasets:

In [ ]:
# your code here
from nanopyx.data.download import ExampleDataManager

edm = ExampleDataManager()
edm.list_datasets()

b) Now let's load the SMLMS2013_HDTubulinAlexa647 using the [class method](https://henriqueslab.github.io/NanoPyx/nanopyx/data/download.html#ExampleDataManager):
```python
get_ZipTiffIterator(dataset_name, as_ndarray=True)
```

In [ ]:
# your code here
image = edm.get_ZipTiffIterator("SMLMS2013_HDTubulinAlexa647", as_ndarray=True)

c) let's look at the image by using stackview

```python
import stackview
stackview.slice(image)
```
  
NOTE: if using colab you will need to add this code before stackview to enable the widget:
```python
from google.colab import output
output.enable_custom_widget_manager()
```

In [ ]:
# your code here
import stackview
stackview.slice(image)

## 1.3 Using NanoPyx eSRRF to generate a super-resolved image

```python
import numpy as np
from nanopyx import eSRRF
sr_image = eSRRF(image, magnification=2)[0] # this function returns a tuple with index 0 corresponding to the image
mean_projection = np.mean(sr_image, axis=0)
```

don't forget to display it with stackview

In [ ]:
# your code here
import numpy as np
from nanopyx import eSRRF
sr_image = eSRRF(image, magnification=2)[0]
stackview.slice(np.mean(sr_image, axis=0))

## 1.4 Let's compare the resolution of the original image and the super-resolved image using image decorrelation analysis

```python
from nanopyx import calculate_decorr_analysis
resolution = calculate_decorr_analysis(image, pixel_size=1, pixel_units="nm")
```

use 100 for the pixel size of the original image and 100/magnification for the eSRRF image

In [ ]:
# your code here
from nanopyx import calculate_decorr_analysis
df_resolution = calculate_decorr_analysis(np.mean(image, axis=0), pixel_size=100)
sr_resolution = calculate_decorr_analysis(np.mean(sr_image, axis=0), pixel_size=50)

print(f"DF resolution: {df_resolution:.2f} px and SR resolution: {sr_resolution:.2f} px")

# 2. Using NanoPyx Liquid Engine to optimize image analysis

NanoPyx uses the Liquid Engine at it's core. This allow us to have multiple implementations of the same algorithm (eSRRF for example) and the Liquid Engine's agent will decide on the best one to use for your input and hardware.


## 2.1 Let's import the 2D convolution method of NanoPyx and run it using a random image

```python   
import numpy as np
from nanopyx.core.transform._le_convolution import Convolution

img = np.random.random((25, 25)).astype(np.float32)
kernel = np.ones((3, 3), dtype=np.float32)
conv = Convolution()
```

In [ ]:
# your code here
import numpy as np
from nanopyx.core.transform._le_convolution import Convolution

img = np.random.random((25, 25)).astype(np.float32)
kernel = np.ones((3, 3), dtype=np.float32)
conv = Convolution()

## 2.2 Benchmarking the different implementations
  
When there are no benchmarks available, the Liquid Engine will use any provided default benchmarks until it has 3 run times on your hardware. If no defaults are provided then it will randomly choosen between all available options.  
  
As part of the Liquid Engine, we have also created a benchmarking function that allows you to benchmark all implementations in your hardware for specific inputs.

```python
for i in range(3):
    conv.benchmark(img, kernel)
```


In [ ]:
# your code here

for i in range(3):
    conv.benchmark(img, kernel)

## 2.3 Benchmark the same method but now using a bigger image and kernel

```python   
import numpy as np

big_img = np.random.random((500, 500)).astype(np.float32)
big_kernel = np.ones((5, 5), dtype=np.float32)

for i in range(3):
    conv.benchmark(big_img, big_kernel)
```

In [ ]:
# your code here
import numpy as np

big_img = np.random.random((500, 500)).astype(np.float32)
big_kernel = np.ones((5, 5), dtype=np.float32)

for i in range(3):
    conv.benchmark(big_img, big_kernel)

## 2.4 Now when calling .run() you will see that the agent will pick the fastest option for each case

```python
out = conv.run(img, kernel)
out2 = conv.run(big_img, big_kernel)
```

In [ ]:
# your code here
out = conv.run(img, kernel)
out2 = conv.run(big_img, big_kernel)

## 2.5 What if you already have benchmarks but not of the specific input parameters that you will be testing?

On those scenarios the Liquid Engine agent employs fuzzy logic to find the closest known example.  
Try running the conv for an image with shape=(150, 150) and for another with shape=(550, 550), they should use the fastest run_type according to the previous images of (100, 100) and (500, 500), respectively.  

In [ ]:
# your code here
out = conv.run(np.random.random((150, 150)).astype(np.float32), np.ones((3, 3), dtype=np.float32))
out2 = conv.run(np.random.random((550, 550)).astype(np.float32), np.ones((5, 5), dtype=np.float32))

## 2.6 Forcing a specific run_type to run

You also have the option of manually forcing the Liquid Engine to use a specific implementation using the `run_type="run_type_name"`optional argument.

```python
out = conv.run(img, kernel, run_type="threaded")
```

In [ ]:
# your code here
out = conv.run(img, kernel, run_type="threaded")

## 2.6.1 You can inspect the class to find which run_types are available

```python
print(conv._run_types.keys())
```

In [ ]:
# your code here
print(conv._run_types.keys())

## 3. Implementing your own method using the Liquid Engine

The Liquid Engine is a standalone package that is pip installable:
```shell
pip install liquid_engine
```

It's a requirement of NanoPyx so you should already have it in your environment.

## 3.1 Let's create our own method using the Liquid Engine but we need to do that inside a .py file and not a jupyter notebook

This is mainly due to how we dynamically check class and methods name so that we can use it for the automatic benchmarking.  

a) Start by creating myliquidengine.py file in the same folder as this notebook or in the collab runtime workspace  
  
b) Add the following code to it:  
```python
import numpy as np
from liquid_engine import LiquidEngine

from skimage.restoration import denoise_nl_means

class MyLiquidEngineClass(LiquidEngine):
    def __init__(self):
        self._designation = "MyLiquidEngineClass"
        super().__init__()

    def run(self, image: np.ndarray, patch_size: int, patch_distance: int, h:float, sigma:float, run_type=None):
        return self._run(image, patch_size=patch_size, patch_distance=patch_distance, h=h, sigma=sigma)

    def _run_ski_nlm_1(self, image, patch_size, patch_distance, h, sigma):
        return denoise_nl_means(image, patch_size=patch_size, patch_distance=patch_distance, h=h, sigma=sigma, fast_mode=True)

    def _run_ski_nlm_2(self, image, patch_size, patch_distance, h, sigma):
        return denoise_nl_means(image, patch_size=patch_size, patch_distance=patch_distance, h=h, sigma=sigma, fast_mode=False)
```

Code explanation:
- run(args): is the call to run the method, any arguments need to be passed here and it should always call _run()
- _run(args): private method defined as part of the Liquid Engine that looks for _run_runtype_name methods and treats them as different implementations to be selected by the agent.
- _run_runtype_name_1, _run_runtype_name_2, ...: different implementations to be selected by the agent, determined by starting it's naming with _run and should always be followed by _runtype_name (whatever you want to call your implementation)  

---

In this example we will be using the non-local means denoising from scikit-image which comes with two different implementations controlled by the `fast_mode` argument.  

  

c) Let's import the class defined in myliquidengine.py file and initialize it:  
```python
from myliquidengine import MyLiquidEngineClass

myle = MyLiquidEngineClass()
```


In [1]:
# your code here
from myliquidengine import MyLiquidEngineClass

myle = MyLiquidEngineClass()

## 3.2 Now let's benchmark it using a random image with shape=(100, 100)

For the remaining parameters, use `patch_size=5`, `patch_distance=10`, `h=0.1`and `sigma=1`   


```python
img = np.random.random((100, 100)).astype(np.float32)
for i in range(3):
    myle.benchmark(img, 5, 10, 0.1, 1)
```

In [ ]:
# your code here
img = np.random.random((100, 100)).astype(np.float32)
for i in range(3):
    myle.benchmark(img, 5, 10, 0.1, 1)

## 3.3 Let's call the run method with the same parameters and check it's selecting the appropriate run_type

In [ ]:
# your code here
out = myle.run(img, 5, 10, 0.1, 1)